In [ ]:
#MAGMA: to identify genetic association between particular cell types and a trait.
#in this sits: https://github.com/neurogenomics/MAGMA_Files_Public/blob/master/metadata.csv
#you can find already processed GWAS results.
#You also need a .csv file with the codes of the GWAS and the name of the study/trait, this will facilitate plotting. 
# R.4.1
rm()
library(ggplot2)
library(reshape2)
library(data.table)
library(knitr)
library(stringr)
#library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)
library(Seurat)
library(SingleCellExperiment)
library(data.table)
 library(dplyr)
library(MAGMA.Celltyping)
library(ggplot2)


In [ ]:
setwd(".../MAGMA")

In [ ]:

storage_dir <- ".../MAGMA/storage"

In [ ]:
NG <-readRDS(".../RData/SEZ_seurat_object.rds")

DefaultAssay(NG) <- "RNA"
Idents(NG) <- NG$cluster

order<-c('16','0','7','8', '19',
            '11','13','14',
         '2','3','12','10',
                        '9')



Idents(NG) <- factor(x = Idents(NG), levels = order)
levels(NG)

In [ ]:
MAGMA_results <- MAGMA.Celltyping::celltype_associations_pipeline(
  magma_dirs = magma_dirs,
  ctd = ctd_2,
  ctd_species = "human", 
  ctd_name = "NSCs_neurons", 
  run_linear = TRUE, 
    run_top10 = TRUE,
    run_conditional = TRUE)


In [ ]:
merged_results <- MAGMA.Celltyping::merge_results(MAGMA_results = MAGMA_results)
knitr::kable(merged_results)
write.csv(merged_results, "GWAS_NSCs_neurons_SZ_BP_AU.csv")

In [ ]:
levels(factor(merged_results$GWAS))

In [ ]:
codes<-read.csv('diseases_code_v2.csv')
merged_results$disease<-codes$diesease[match(merged_results$GWAS, codes$code)]
#merged_results

In [ ]:
merged_results$disease<-factor(merged_results$disease, levels=c('SZ-Ripke-2014_v1',
                                                               'BP-Neale-2017',
                                                               'Autism-PCG-IPSYCH-2017'))
levels(merged_results$disease)

In [ ]:
order=levels(NG)
merged_results$Celltype_id<-factor(merged_results$Celltype_id,levels=order)
levels(merged_results$Celltype_id)

In [ ]:
merged_results_linear<-merged_results[merged_results$EnrichmentMode == 'Linear',]

In [ ]:
min<-min(merged_results_linear$FDR)
max<-max(merged_results_linear$FDR)

In [ ]:
tiff("heatmap_diseases_NSC_neurons_sig_005_SZ_BP_AU_2.tiff", res = 400, units = "in", width = 7, height = 1.5)

ggplot(merged_results_linear, aes(x=factor(Celltype_id, levels=order), y=disease, fill= FDR))+  
  geom_tile(color = "black")+
    scale_fill_gradient(low="#67001F", high="#E7E1EF", limits=c(min,0.05),na.value = "white")+

    theme(strip.text = element_text(size=10),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
    panel.background = element_blank(), axis.line = element_line(colour = "white"),axis.text=element_text(size=10,color="black"),
          text=element_text( face = "bold" ,family="Arial", size=10),strip.background = element_blank(), axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
        axis.text.y = element_text(size=10),
         axis.title.x = element_blank(),axis.ticks.x=element_blank(),axis.ticks.y=element_blank(),
          axis.title.y = element_blank())
dev.off()

In [ ]:
magma_dirs<-c("MAGMA_Files/ieu-a-22.vcf_cleaned_GRCh37.txt.35UP.10DOWN/",
             "MAGMA_Files/ukb-a-525.vcf_cleaned_GRCh37.txt.35UP.10DOWN/",
               "MAGMA_Files/ieu-a-1185.vcf_cleaned_GRCh37.txt.35UP.10DOWN")

In [ ]:
#convert my seurat to ctd
library(EWCE)
#obtain expression matrix pf seurat

matrix<-as.matrix(GetAssayData(NG, assay = "integrated"))

#matrix<-t(matrix)#cells should be rows
var_genes<-rownames(matrix)


logcounts <- as.matrix(GetAssayData(NG,slot = "data")[var_genes,])
meta <- as.data.frame(NG@meta.data)
fdata <- data.frame(primerid=rownames(NG), stringsAsFactors = F)
annotLevels1<-as.character(meta$cluster)
annotLevels2<-as.character(meta$hier_cluster)

annotLevels_l<-list(annotLevels1,annotLevels2)#


ctd<-generate_celltype_data(logcounts,groupName = "NSCs_neurons",
  annotLevels_l,
  savePath=storage_dir)

ctd_2 <- EWCE::load_rdata(ctd)

In [ ]:
MAGMA_results <- MAGMA.Celltyping::celltype_associations_pipeline(
  magma_dirs = magma_dirs,
  ctd = ctd_2,
  ctd_species = "human", 
  ctd_name = "NSCs_neurons", 
  run_linear = TRUE, 
    run_top10 = TRUE,
    run_conditional = TRUE)


merged_results <- MAGMA.Celltyping::merge_results(MAGMA_results = MAGMA_results)
knitr::kable(merged_results)
write.csv(merged_results, "GWAS_NSCs_neurons_SZ_BP_AU.csv")

levels(factor(merged_results$GWAS))

codes<-read.csv('diseases_code_v2.csv')
merged_results$disease<-codes$diesease[match(merged_results$GWAS, codes$code)]
#merged_results

merged_results$disease<-factor(merged_results$disease, levels=c('SZ-Ripke-2014_v1',
                                                               'BP-Neale-2017',
                                                               'Autism-PCG-IPSYCH-2017'))
levels(merged_results$disease)

order=levels(NG)
merged_results$Celltype_id<-factor(merged_results$Celltype_id,levels=order)
levels(merged_results$Celltype_id)

merged_results_linear<-merged_results[merged_results$EnrichmentMode == 'Linear',]

min<-min(merged_results_linear$FDR)
max<-max(merged_results_linear$FDR)

tiff("heatmap_diseases_NSC_neurons_sig_005_SZ_BP_AU_2.tiff", res = 400, units = "in", width = 7, height = 1.5)

ggplot(merged_results_linear, aes(x=factor(Celltype_id, levels=order), y=disease, fill= FDR))+  
  geom_tile(color = "black")+
    scale_fill_gradient(low="#67001F", high="#E7E1EF", limits=c(min,0.05),na.value = "white")+

    theme(strip.text = element_text(size=10),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
    panel.background = element_blank(), axis.line = element_line(colour = "white"),axis.text=element_text(size=10,color="black"),
          text=element_text( face = "bold" ,family="Arial", size=10),strip.background = element_blank(), axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
        axis.text.y = element_text(size=10),
         axis.title.x = element_blank(),axis.ticks.x=element_blank(),axis.ticks.y=element_blank(),
          axis.title.y = element_blank())
dev.off()